### **Load Environment variables from .env file**

In [15]:
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from openai import AzureOpenAI
from dotenv import load_dotenv
import os

In [9]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_DEPLOYMENT_NAME = os.getenv("OPENAI_DEPLOYMENT_NAME")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME")
OPENAI_DEPLOYMENT_VERSION = os.getenv("OPENAI_DEPLOYMENT_VERSION")

OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME = os.getenv("OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME")
OPENAI_ADA_EMBEDDING_MODEL_NAME = os.getenv("OPENAI_ADA_EMBEDDING_MODEL_NAME")

client = AzureOpenAI(
  azure_endpoint = OPENAI_DEPLOYMENT_ENDPOINT, 
  api_key=OPENAI_API_KEY,  
  api_version="2023-05-15"
)

In [10]:
embeddings = AzureOpenAIEmbeddings(
    deployment=OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME,
    model=OPENAI_ADA_EMBEDDING_MODEL_NAME,
    openai_api_base=OPENAI_DEPLOYMENT_ENDPOINT,
    chunk_size = 1)


c:\Users\dschlesinger\code\ongoing\workshops\AOAI-workshop\.venv\lib\site-packages\langchain\embeddings\azure_openai.py:101: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://aidemos-workshop.openai.azure.com/ to https://aidemos-workshop.openai.azure.com//openai.
  warnings.warn(
c:\Users\dschlesinger\code\ongoing\workshops\AOAI-workshop\.venv\lib\site-packages\langchain\embeddings\azure_openai.py:108: UserWarning: As of openai>=1.0.0, if `deployment` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
c:\Users\dschlesinger\code\ongoing\workshops\AOAI-workshop\.venv\lib\site-packages\langchain\embeddings\azure_openai.py:116: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specif

#### **Run ONLY ONCE to create the embeddings - Split text into chunks** 

In [ ]:
fileName = "./data/fabric-data-engineering.pdf"
loader = PyPDFLoader(fileName)
pages = loader.load_and_split()
print("Number of pages: ", len(pages))

Number of pages:  292


#### **Run ONLY ONCE to create the embeddings - Create embeddings and save to FAISS**

In [ ]:
db = FAISS.from_documents(documents=pages, embedding=embeddings)
# save the FAISS index to disk
db.save_local("./dbs/documentation/faiss_index")

In [20]:
client = AzureOpenAI(
  azure_endpoint = OPENAI_DEPLOYMENT_ENDPOINT, 
  api_key=OPENAI_API_KEY,  
  api_version="2023-05-15"
)

def call_openAI(text):
    # prepare prompt
    messages = [{"role": "system", "content": "You are a HELPFUL assistant answering users questions. Answer in a clear and concise manner."},
                {"role": "user", "content": text}]
    response = client.chat.completions.create(
        model=OPENAI_DEPLOYMENT_NAME,
        messages = messages,
        temperature=0.7,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    return response.choices[0].message.content

#### **Initialize retrieval API WITH your data **

In [23]:
# load the vector store to memory
vectorStore = FAISS.load_local("./dbs/documentation/faiss_index", embeddings)
retriever = vectorStore.as_retriever(search_type="similarity", search_kwargs={"k": 2})  # returns 2 most similar vectors/documents
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=False)

#### **Ask questions**

In [24]:
qa({"query": "Can I use PowerBI datamart for 110 GB data volume? Explain which other options are available for data volumes of 110 GB?"})

{'query': 'Can I use PowerBI datamart for 110 GB data volume? Explain which other options are available for data volumes of 110 GB?',
 'result': ' No, PowerBI datamart is only suitable for data volumes up to 100 GB. For data volumes of 110 GB, you can use a data warehouse or a lakehouse in Microsoft Fabric. \n\nData warehouse is suitable for structured data, and is ideal for SQL engineers and data warehouse developers. It is organized by databases, schemas, and tables, and read operations can be performed using Spark and T-SQL. Write operations can be performed using T-SQL.\n\nLakehouse is suitable for unstructured, semi-structured, and structured data, and is ideal for data engineers and data scientists. It is organized by folders and files, databases, and tables, and read operations can be performed using Spark, T-SQL, and Power BI. Write operations can be performed using Spark (Scala, PySpark, Spark SQL, R) and T-SQL. \n\nBoth data warehouse and lakehouse can handle unlimited data v

In [25]:
qa({"query": "What are the steps to load a CSV file to a delta table in Microsoft Fabric?"})

{'query': 'What are the steps to load a CSV file to a delta table in Microsoft Fabric?',
 'result': ' The steps to load a CSV file to a delta table in Microsoft Fabric are:\n1. Select Synapse Data Engineering experience in Microsoft Fabric.\n2. Make sure you are in the desired workspace or select/create one.\n3. Select the Lakehouse icon under the New section in the main image.\n4. Upload the CSV file to the Lakehouse.\n5. Convert the file to a Delta table. \n6. Generate a Dataset and create a Power BI report.'}